In [ ]:
import pandas as pd
import numpy as np
from scipy import interpolate
from scipy.optimize import fsolve
import plotly.graph_objects as go
import plotly.express as px

## Loading Data

In [ ]:
def load_calibration_data(filepath):
    """Loads UFR tilt calibration data from file.

    Loads UFR tilt calibration data from .csv file and 
    returns the data as a pandas dataframe.

    Arguments (Required)
    --------------------
    filename : str
        Path to calibration file in form '/..../calibration_file.csv'

    Returns
    -------
    pandas dataframe
        5-column dataframe containing:
            theta_x - Optics tilt angle around x-axis (degrees).
            theta_y - Optics tilt angle around y-axis (degrees).
            qpd_x   - Signal from QPD: beam position in x-direction.
                      Normalised by qpd_sum so in range [-1 1].
            qpd_y   - Signal from QPD: beam position in y-direction.
                      Normalised by qpd_sum so in range [-1 1].
            qpd_sum - Signal from QPD: beam intensity.

    Raises
    ------
    FileNotFoundError
        Raised if filename points 
    """
    col_names = ['theta_x', 'theta_y', 'qpd_x', 'qpd_y', 'qpd_sum']
    df = pd.read_csv(filename, names=col_names)
    
    angular_range = (df['theta_x'].min(),
                     df['theta_x'].max(),
                     df['theta_y'].min(),
                     df['theta_y'].max())
    return df, angular_range

filename = '../data/2020-03-10_QPD-Tilt-Calibration_Test1.csv'
calibration_data, calibration_data_angular_range = load_calibration_data(filename)
print(calibration_data)
print(calibration_data_angular_range)

## Surface Fitting
Fit 2D Functions of form $q_x=f(\theta_x, \theta_y)$ and $q_y=f(\theta_x, \theta_y)$

e.g. using 3rd order polynomials:

$q_x = a_1.\theta_x^3 + a_2.\theta_y^3 + a_3.\theta_x^2 + a_4.\theta_y^2 + a_5.\theta_x + a_6.\theta_y + a_7$

$q_y = b_1.\theta_x^3 + b_2.\theta_y^3 + b_3.\theta_x^2 + b_4.\theta_y^2 + b_5.\theta_x + b_6.\theta_y + b_7$

In [ ]:
def surface_fitting(x, y, z):
    """Fits 3rd order polynomial surface to z(x, y).

    Fits 3rd order polynomial function of form zfit = f(x) + g(y) to 
    scattered or gridded data of form z(x, y).

    Arguments (Required)
    --------------------
    x, y, z : array_like
              x, y, and z co-ordinates of the data points.
              May be 1D (e.g. scattered data) or 2D (e.g. gridded data).
              x, y and z must have same dimensions.

    Returns
    -------
    numpy.ndarray
        1D 7-element array containing the coefficients of:
            X^3, Y^3, X^2, Y^2, X, Y, const

    Raises
    ------
    LinAlgError
        Raised if computation of surface fit does not converge.
        Indicative of poor calibration data.
    """

    assert x.shape == y.shape, 'x and y arrays must have same dimensions'
    assert z.shape == x.shape, 'z array must have same dimension as x and y arrays'

    X = x.flatten()
    Y = y.flatten()
    Z = z.flatten()

    A = np.array([X**3, Y**3, X**2, Y**2, X, Y, X*0+1]).T

    coeff, residuals, rank, s = np.linalg.lstsq(A, Z, rcond=None)
    
    return coeff

In [ ]:
c = np.zeros([2, 7])
c[0, :] = surface_fitting(calibration_data['theta_x'].to_numpy(), 
                          calibration_data['theta_y'].to_numpy(),
                          calibration_data['qpd_x'].to_numpy())
c[1, :] = surface_fitting(calibration_data['theta_x'].to_numpy(),
                          calibration_data['theta_y'].to_numpy(),
                          calibration_data['qpd_y'].to_numpy())
print(c)

### Plot the fitted surfaces

In [ ]:
def poly2Dreco(X, Y, c):
    """Evaluates 3rd order polynomial from coefficients

    Evaluates 3rd order polynomial function of form zfit = f(x) + g(y) to 
    scattered or gridded data of form z(x, y).

    Arguments (Required)
    --------------------
    X, Y : array_like
        Locations to evaluate the function: x and y co-ordinates.
        Can be 1D (e.g. scattered data) or 2d (e.g. gridded data)
    c : numpy.ndarray
        1D 7-element array containing the coefficients of:
            X^3, Y^3, X^2, Y^2, X, Y, const

    Returns
    -------
    numpy.ndarray
        Values of function at co-ordinates (x, y).
        Dimensions same as X, Y.
    """

    assert X.shape == Y.shape, 'x and y arrays must have same dimensions'
    assert c.shape == (7,), 'Incorrect dimensions for coefficent array'

    return (c[0]*X**3 + c[1]*Y**3 + c[2]*X**2 + c[3]*Y**2 + c[4]*X + c[5]*Y + c[6])

def plot_fitted_surface(x_raw, y_raw, z_raw, c, plot_opts):
    
    grid_x, grid_y = define_grid(x_raw, y_raw, 100)

    zfit = poly2Dreco(grid_x, grid_y, c)

    fig = go.Figure()

    fig.add_trace(go.Scatter3d(x=x_raw, y=y_raw, z=z_raw, mode='markers'))


    fig.add_trace(go.Surface(z=zfit, x=grid_x, y=grid_y))
                    
    fig.update_layout(scene = plot_opts)

    fig.update_layout(
    autosize=False,
    width=800,
    height=800)
    
    fig.show()

def define_grid(x_raw, y_raw, nxy):
    """Creates gridded co-ordinates that span range of scattered co-ordinates.

    Generates a ny [#rows] x nx [#cols] grid of (x, y) co-ordinates from scattered data 
    (x_raw, y_raw) such that the all scattered points are contained within the grid.

    x ranges from min(x_raw) to max(x_raw)
    y ranges from min(y_raw) to max(y_raw)

    nxy can be a single integer e.g. 100 in which case nx = ny = nxy
                              or
    nxy can be a tuple of two ints in format (nx, ny)

    Arguments (Required)
    --------------------
    x_raw, y_raw : array_like
        1D or 2D array of x and y co-ordinates. Max and min of each array used to
        define limits of the generated grids

    nxy : int or tuple of 2 ints
        int: equal number of points in x and y directions, nx = ny.
        tuple of 2 ints: nx = nxy[0]
                         ny = nxy[1]

    Returns
    -------
    numpy.ndarray
        Arrays containing grid of x co-ordinates and y co-ordinates
    """
    assert type(nxy) == int or type(nxy) == tuple, 'nxy must be int or (int, int)'
    if type(nxy) == tuple:
        assert len(nxy) == 2, 'nxy as tuple must contain 2 integer values only'
        assert all(type(i) == int for i in nxy), 'all elements of nxy must be of type int'
        nx = nxy[0]
        ny = nxy[1]
        assert nx > 0, 'nx must be positive'
        assert ny > 0, 'ny must be positive'
    else: # type(nxy) == int
        assert nxy > 0, 'nxy must be positive'
        nx = nxy
        ny = nxy

    # Calculate upper and lower extent of grid   
    x_min = x_raw.min()
    x_max = x_raw.max()
    y_min = y_raw.min()
    y_max = y_raw.max()

    # Define step size (complex number means 'this many steps' in np.mgrid)
    x_step = nx * 1j
    y_step = ny * 1j

    # Generate grid
    grid_x, grid_y = np.mgrid[x_min:x_max:x_step, y_min:y_max:y_step]

    return grid_x, grid_y

Plot variation of QPD x-position with $\theta_x$ and $\theta_y$

In [ ]:
plot_opts = dict(
            xaxis_title='theta_x',
            yaxis_title='theta_y',
            zaxis_title='QPD x')
plot_fitted_surface(calibration_data['theta_x'],
                    calibration_data['theta_y'],
                    calibration_data['qpd_x'],
                    c[0, :],
                    plot_opts)

Plot variation of QPD x-position with $\theta_x$ and $\theta_y$

In [ ]:
plot_opts = dict(
            xaxis_title='theta_x',
            yaxis_title='theta_y',
            zaxis_title='QPD y')
plot_fitted_surface(calibration_data['theta_x'],
                    calibration_data['theta_y'],
                    calibration_data['qpd_y'],
                    c[1, :],
                    plot_opts)

# Calibration Procedure


## Numerical Calculation

In [ ]:
def calc_angles_from_qpd_values(c, qpd_pos, calculation_grid_dimensions):
    """Calibration function - converts QPD position to angles 

    Converts QPD position (qpd_x, qpd_y) to angles (theta_x, theta_y)

    Calculates angles theta_x and theta_y that satisfy:
        [A] fitted surface qpd_x(theta_x, theta_y) == qpd_pos[0]
        [B] fitted surface qpd_y(theta_x, theta_y) == qpd_pos[1]
    Each takes the form of a line in the (theta_x, theta_y) plane
    
        Each of [A] and [B] are calculated numerically as the intersection between:
            [i] the polynomial surface z = poly2Dreco(theta_x, theta_y, c)
                where 'c' are the parameters identified in surface_fitting()
            [ii] the plane z = <qpd value>

    The 'common intersection' between the two calculated lines in the
    (theta_x, theta_y) plane is calculated. This identifies the pair of 
    (theta_x, theta_y) values which satisfy equations [A] and [B].

    Arguments (Required)
    --------------------
    c : numpy.ndarray
        1D 7-element array containing the coefficients of:
            X^3, Y^3, X^2, Y^2, X, Y, const
    qpd_pos : 2-element tuple of float
        Containing QPD position for evaluation (qpd_x, qpd_y).
    calculation_grid_dimensions : dict(tx_vec, ty_vec)
        Dictionary containing tx_vec and ty_vec
            tx_vec, ty_vec : numpy.ndarray
                1D array containing grid of angles theta_x and theta_y
                used in calculation

    Returns
    -------
    2-element tuple of float
        <arg1 description>
    """

    # [1] Define grid of (theta_x, theta_y)
    grid_tx, grid_ty = np.meshgrid(calculation_grid_dimensions['tx_vec'], calculation_grid_dimensions['ty_vec'])

    # [2] Identify as True the (theta_x, theta_y) values which generate QPD position 'qpd_pos'
    is_common_intersection = calc_common_intersection(grid_tx, grid_ty, c, qpd_pos)
    
    # [3] Extract mean theta_x and theta_y from True values in is_common_intersection array
    thetas = calc_angles_from_common_intersection(grid_tx, grid_ty, is_common_intersection)

    return thetas

def calc_common_intersection(grid_tx, grid_ty, c, qpd_pos):

    intersection_array = np.zeros((2,) + grid_tx.shape) # initialise

    for i in range(0, 2): # For qpd_x and qpd_y,
        # Calculate plane-surface intersection
        z_qpd = poly2Dreco(grid_tx, grid_ty, c[i, ]) - qpd_pos[i] # z_qpd_x == 0 where qpd_x == qpd_pos[0]
        is_intersection = np.abs(z_qpd - 0) < 0.01

        intersection_array[i,] = is_intersection

    # Calculate common intersection (i.e. find theta_x and theta_y position)
    is_common_intersection = np.all(intersection_array, axis=0)
    
    return is_common_intersection

def calc_angles_from_common_intersection(grid_tx, grid_ty, is_common_intersection):

    theta_x = np.mean(grid_tx[is_common_intersection])
    theta_y = np.mean(grid_ty[is_common_intersection])

    thetas = (theta_x, theta_y)

    return thetas

In [ ]:
def verify_qpd_from_angles(thetas, c):

    x_qpd_chk = poly2Dreco(thetas[0], thetas[1], c[0, ])
    y_qpd_chk = poly2Dreco(thetas[0], thetas[1], c[1, ])

    print('verification qpd position: ' + str([x_qpd_chk, y_qpd_chk]))

Demonstrate QPD --> angle calculation:

In [ ]:
qpd_pos = np.array([0, 0.09])

calculation_grid_dimensions = dict(tx_vec=np.arange(-0.5, 0.5, 0.01),
                                   ty_vec=np.arange(-0.4, 0.9, 0.01))

thetas_numerical = calc_angles_from_qpd_values(c, qpd_pos, calculation_grid_dimensions)

print('input qpd position: ' + str(qpd_pos))
print('calculated angles are ' + str(thetas_numerical) + ' degrees')

verify_qpd_from_angles(thetas_numerical, c)

## Produce Calibration File
### Iterate through QPD positions

In [ ]:
def iterate_qpd_positions(grid_qx, grid_qy):
    tx_array = np.zeros_like(grid_qx)
    ty_array = np.zeros_like(grid_qx)
    
    for i, j in np.ndindex(grid_qx.shape):

        qpd_pos = np.array([grid_qx[i, j], grid_qy[i, j]])

        t = calc_angles_from_qpd_values(c, qpd_pos, calculation_grid_dimensions)
        
        tx_array[i, j] = t[0]
        ty_array[i, j] = t[1]
        
    print('done')

    return tx_array, ty_array

In [ ]:
# iterate through QPD positions
qx_vec = np.arange(-1, 1, 0.1)
qy_vec = np.arange(-1, 1, 0.1)
grid_qx, grid_qy = np.meshgrid(qx_vec, qy_vec)

tx_array, ty_array = iterate_qpd_positions(grid_qx, grid_qy)

In [ ]:
fig = px.scatter(x=grid_qx.flatten(), y=grid_qy.flatten(), color=tx_array.flatten())
fig.update_layout(xaxis_title='QPD x',
                  yaxis_title='QPD y')
fig.show()

Fit surface to calibration data:


In [ ]:
d = np.zeros([2, 7])
d[0, :] = surface_fitting(grid_qx.flatten(), grid_qy.flatten(), tx_array.flatten())
d[1, :] = surface_fitting(grid_qx.flatten(), grid_qy.flatten(), ty_array.flatten())
print(d)

Plot variation of $\theta_x$ with $q_x$ and $q_y$

In [ ]:
plot_opts = dict(
            xaxis_title='QPD x',
            yaxis_title='QPD y',
            zaxis_title='theta_x')
plot_fitted_surface(grid_qx.flatten(), grid_qy.flatten(), tx_array.flatten(), d[0, :], plot_opts)

Plot variation of $\theta_y$ with $q_x$ and $q_y$

In [ ]:
plot_opts = dict(
            xaxis_title='QPD x',
            yaxis_title='QPD y',
            zaxis_title='theta_y')
plot_fitted_surface(grid_qx.flatten(), grid_qy.flatten(), ty_array.flatten(), d[1, :], plot_opts)